# Final Project
____Group 8____

Mandy Gu

Tejasvini Karunakarbabu

Mayank Mani

Willa Yu

__Import all packages needed__

In [1]:
import numpy as np
import mysql.connector
from sqlalchemy import create_engine
import pymysql
import json
import time
from datetime import date
from dateutil.parser import parse
from datetime import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup 
from tqdm import notebook
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import datetime
import re
from datetime import timedelta
import glob
import os

__1. Store the to-be-scraped pages and other necassary information.__ 

Create the company list

In [ ]:
companies = ['FB', 'EXPEDIA', 'COSTCO', 'GOOGL', 'ADBE', 'SBUX', 'TSLA', 'NFLX', 'AAPL', 'AMZN']

Since Nasdaq uses a dynamic web structure, launch the webdriver to get the page contents

In [ ]:
driver = webdriver.Chrome()

Create the directory for storing all the urls

In [6]:
test1 = os.getcwd()+'\\news'
if not os.path.exists(test1):
    os.makedirs(test1)

Download all the pages that we need using webdriver, store them in the 'news' directory

In [ ]:
company_list_t = []
current_time_list_t = []

for company in notebook.tqdm(companies):
    # create a 50-page list for testing
    url_list = [str('https://www.nasdaq.com/search?q='+ company + '&page='+ str(i)+ '&sort_by=recent') for i in range(1,51)]
    page_num = 1
    # get the time the selenium is started
    current_time = datetime.datetime.now()
    
    # download the page
    for url in notebook.tqdm(url_list):
        driver.get(url)
        pagesource = driver.page_source
        time.sleep(2)
        file_name = str(company) + ' ' + str(page_num) + '.txt'
        name_wdir = os.path.join(test1, file_name)
        file = open(name_wdir, 'w',  encoding = "utf-8")
        file.write(str(pagesource))
        file.close()
        page_num = page_num + 1
        company_list_t.append(company)
        current_time_list_t.append(current_time)


Save the company and current_time to a csv for future use (and read it)

In [ ]:
news_cnt = pd.DataFrame({
    'company': company_list_t,
    'current_time': current_time_list_t,
    })
name_wdir_csv1 = os.path.join(test1, 'news_company_n_time.csv')
news_cnt.to_csv(name_wdir_csv1, index = False)

# read the csv with company name and current time 
news_cnt1 = pd.read_csv(name_wdir_csv1)

Get the file names of the scraped txts

In [ ]:
page_list = []
for company in companies:
    page_list = list(page_list + [company + ' ' + str(i) + '.txt' for i in range(1,51)])

Create the function to calculate publish day

In [ ]:
def day_calculator(time):
    '''
    get the date of news
    ''' 
    if bool(re.search('DAY', time.text, re.IGNORECASE)):
        day_delta = int((re.findall('([0-9]+)', time.text)[0]))
        day_diff = timedelta(days = day_delta)
        press_day = (current_time - day_diff).date()
    elif bool(re.search('HOUR', time.text, re.IGNORECASE)):
        hour_delta = int((re.findall('([0-9]+)', time.text)[0]))
        hour_diff = timedelta(hours = hour_delta)
        press_day = (current_time - hour_diff).date()
    elif bool(re.search('MIN', time.text, re.IGNORECASE)):
        press_day = (current_time).date()
    elif time.text == '':
        press_day = ''
    else:
        press_day = parse(time.text).date()
    return press_day
day_calculate = lambda x: day_calculator(x)

__2. Scrape news headlines, related company name, published time, and publisher from the stored text files.__

In [ ]:
company_list = []
titles_list = []
publisher_list = []
page_url_list = []
url_list = []
release_date_list = []
weekday_list = []
weekday = []
try_list = []
titles_count = []

for i in notebook.tqdm(range(len(page_list))):
    # Open the file
    name_wdir2 = os.path.join(test1, page_list[i])
    page_content = open(name_wdir2, "r", encoding="utf-8").read()
    # Assign a list to the file
    soup = BeautifulSoup(page_content)
    
    try_list.append(page_list[i])
    # get the searched company neme for this page
    company_name = re.sub(r'([A-Z]+)( [0-9]+.*)', r'\1', page_list[i])
    # get the list of titles
    title_container = soup.find_all(class_ = 'search-result__link')
    titles = [x.text for x in title_container]
    titles_count.append(len(titles))
    # get the list of news release time: e.g. '1 minute ago', 26 DAYS AGO'
    current_time = parse(news_cnt1.loc[:,'current_time'][i//50])
    press_time = soup.find_all(class_ = 'search-result__date')
    release_date = list(map(day_calculate, press_time))
    for x in release_date:
        if x == '':
            weekday.append('')
        else:
            weekday.append(x.isoweekday()) 
#     weekday = [x.isoweekday() for x in release_date]
    # get the publisher
    footer = soup.find_all(class_ = 'search-result__footer')
    publisher = []
    for content in footer:
        if content.find(class_ = 'search-result__topic') is None:
            publisher.append('No publisher')
        else:
            publisher.append(content.find(class_ = 'search-result__topic').text)
    # get the url
    page_url = ['https://www.nasdaq.com' + x["href"] for x in title_container]

        
    # create a list for each page      
    company_list = list(company_list + [company_name for i in range(len(titles))]) # every page contains 10 news
    titles_list = list(titles_list + titles)
    release_date_list = list(release_date_list + release_date)
    publisher_list = list(publisher_list + publisher)
    page_url_list = list(page_url_list + page_url)
 

In [ ]:
# create a dataframe 
news_df = pd.DataFrame({
    'company': company_list,
    'titles': titles_list,
    'release_date': release_date_list,
    'weekday': weekday,
    'publisher': publisher_list,
    'page_url': page_url_list
})
news_df

In [ ]:
# store the dataframe as csv to the directory
name_wdir3 = os.path.join(test1, "news_list.csv")
news_df.to_csv(name_wdir3, index = False)

In [ ]:
# read the stored dataframe
csv_path = test1 + "\\news_list.csv"
news_df2 = pd.read_csv(csv_path)
news_df2

__3. Get the article contents and 'other topics' in the news as weight__ 

In [ ]:
full_list = news_df.copy()
page_url_sub = full_list.loc[:, 'page_url']

In [ ]:
driver = webdriver.Chrome()

In [ ]:
content_all_news = []
weight_all_news = []
texter = lambda x: x.text

for url in notebook.tqdm(page_url_sub):
    driver.get(url)
    par_str = ''
    for par in driver.find_elements_by_class_name('body__content')[:-3]:
        par_str = par_str + '\n' + par.text
    content_all_news.append(par_str)
    weight_all_news.append(list(map(texter, driver.find_elements_by_class_name('topics-in-this-story__symbol'))))
    time.sleep(1.5)

In [ ]:
weight_list = pd.DataFrame({
    'article': content_all_news,
    'weights': weight_all_news
})

In [ ]:
full_list_wweight = pd.concat([full_list, weight_list],axis = 1)
full_list_wweight.to_excel('full_list_wweight.xlsx', index = False)

__4. Get the 10 companies' stock price via API__

In [9]:
companies_sym = ['FB', 'EXPE', 'COST', 'GOOGL', 'ADBE', 'SBUX', 'TSLA', 'NFLX', 'AAPL', 'AMZN', 'NDAQ']

In [22]:
# api_df = pd.DataFrame()
cwd = os.getcwd()
api_path = cwd + '\\api'
if not os.path.exists(api_path):
    os.makedirs(api_path)

# fetch all the apis    
for company_sym in notebook.tqdm(companies_sym):
    api_url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=" + company_sym +"&apikey=XOJE92KV355K5ZXX"
    api_response = requests.request("GET", api_url)
    api_dict = api_response.json()
    api_df = pd.DataFrame(api_dict['Time Series (Daily)']).T
    api_df['company'] = company_sym
    api_file_name = api_path + '\\' + company_sym + '_api.xlsx'
    api_df.to_excel(api_file_name, index = True)
    time.sleep(12)

In [26]:
# Pls ensure you have no other xlsx files in this directory
api_dfs = pd.DataFrame()
for filename in glob.glob(api_path + '\\*.xlsx'):
    api_read_xlsx = pd.read_excel(filename)
    api_dfs = pd.concat([api_dfs, api_read_xlsx])
api_dfs = api_dfs.rename(columns={'Unnamed: 0': 'date'})

In [33]:
api_full_path = api_path + '\\api_full.xlsx'
api_dfs.to_excel(api_full_path, index = False)

In [34]:
api_dfs

,date,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,company
0,2020-03-10,277.14,286.44,269.37,285.34,285.34,70721316,0.0,1.0,AAPL
1,2020-03-09,263.75,278.09,263.00,266.17,266.17,71686208,0.0,1.0,AAPL
2,2020-03-06,282.00,290.82,281.23,289.03,289.03,56544246,0.0,1.0,AAPL
3,2020-03-05,295.52,299.55,291.41,292.92,292.92,46893219,0.0,1.0,AAPL
4,2020-03-04,296.44,303.40,293.13,302.74,302.74,54794568,0.0,1.0,AAPL
...,...,...,...,...,...,...,...,...,...,...
95,2019-10-22,254.32,258.33,250.85,255.58,255.58,4625095,0.0,1.0,TSLA
96,2019-10-21,258.33,259.50,250.18,253.50,253.50,5020339,0.0,1.0,TSLA
97,2019-10-18,260.70,262.80,255.10,256.95,256.95,5720888,0.0,1.0,TSLA
98,2019-10-17,262.50,264.78,260.17,261.97,261.97,4779043,0.0,1.0,TSLA


__5.Create a SQL database for the dataframes__

dataframes including news_full_list_with_weight and api_full_list

In [26]:
news_fulllist = pd.read_excel('full_list_wweight.xlsx', ncoding = "cp1252")
api_fulllist = pd.read_excel('api_full.xlsx', ncoding = "cp1252")

In [4]:
news_fulllist.head()

,company,titles,release_date,weekday,publisher,page_url,article,other companies
0,AAPL,Relative Strength Alert For Apple,2/27/2020,4.0,BNK Invest,https://www.nasdaq.com/articles/relative-stren...,\nThe DividendRank formula at Dividend Channel...,['AAPL']
1,AAPL,Why Computer Stocks Fell Today,2/27/2020,4.0,The Motley Fool,https://www.nasdaq.com/articles/why-computer-s...,\nWhat happened\nShares of prominent computer-...,"['AMD', 'AAPL', 'MSFT', 'INTC']"
2,AAPL,2 Key Trends to Watch in Music Streaming,2/27/2020,4.0,The Motley Fool,https://www.nasdaq.com/articles/2-key-trends-t...,\nStreaming accounted for nearly 80% of U.S. m...,"['SPOT', 'AAPL', 'GOOGL', 'SIRI', 'GOOG']"
3,AAPL,Apple (AAPL) Down 9.8% Since Last Earnings Rep...,2/27/2020,4.0,Zacks,https://www.nasdaq.com/articles/apple-aapl-dow...,\nIt has been about a month since the last ear...,[]
4,AAPL,Apple's Coronavirus Weakness Could Mean Invest...,2/27/2020,4.0,The Motley Fool,https://www.nasdaq.com/articles/apples-coronav...,\nWith worries about the SARS-CoV-2 virus at f...,"['AAPL', 'BRK.A', 'BRK.B']"


__Create the database for news__

Since there are many 'other companies' in one news, in order to store it in MySQL, we need to pick only a subset of 'other companies' and split them into several columns.

Here we choose __4 as the number of splitted columns__ since the average number of 'other companies' per news is approximately 3.5.

In [85]:
lenlist = []
for i in news_fulllist['other companies'].str.strip('[').str.strip(']').str.replace("'","").str.replace(' ','').str.replace('\n','').str.split(',').tolist():
    if type(i) != float:
        lenlist.append(len(i))
        # avg number of other companies
        sum(lenlist)/len(lenlist)

3.493854138213603

In [46]:
new_list = []
for i in news_fulllist['other companies'].str.strip('[').str.strip(']').str.replace("'","").str.replace(' ','').str.replace('\n','').str.split(',').tolist():
    if type(i) == float:
        new_list.append(['None'])
    elif i[0] == '':
        new_list.append(['None'])
    else:
        new_list.append(i[0:4])

In [47]:
other_companies = pd.DataFrame(new_list, columns = ['other company 1', 'other company 2', 'other company 3', 'other company 4'])

In [48]:
news_fulllist_wide = pd.concat([news_fulllist.drop('other companies', 1), other_companies], axis = 1)

Transform the data type of release_date, weekday

In [49]:
news_fulllist_wide['release_date'] = pd.to_datetime(news_fulllist_wide['release_date']).astype(str)
news_fulllist_wide['weekday'] = news_fulllist_wide['weekday'].astype('Int64')

In [171]:
news_fulllist_wide.head(2)

,company,titles,release_date,weekday,publisher,page_url,article,other company 1,other company 2,other company 3,other company 4
0,AAPL,Relative Strength Alert For Apple,2020-02-27,4.0,BNK Invest,https://www.nasdaq.com/articles/relative-stren...,\nThe DividendRank formula at Dividend Channel...,AAPL,None,None,None
1,AAPL,Why Computer Stocks Fell Today,2020-02-27,4.0,The Motley Fool,https://www.nasdaq.com/articles/why-computer-s...,\nWhat happened\nShares of prominent computer-...,AMD,AAPL,MSFT,INTC


Drop the duplicated news that have the same primary key (company, title)

In [50]:
news_fulllist_wide.drop_duplicates(subset=['company','titles', 'release_date'],keep='first',inplace = True)

Calculate the max length of titles

In [138]:
max(news_fulllist_wide['titles'].apply(lambda x: (len(x))))

251

In [202]:
max(news_fulllist_wide['page_url'].apply(lambda x: (len(x))))

124

In [51]:
max(news_fulllist_wide['company'].apply(lambda x: (len(x))))

7

In [53]:
news_fulllist_wide = news_fulllist_wide.where((pd.notnull(news_fulllist_wide)), None)

In [54]:
# Connect to mysql and specifically the database named eBay
mydb = mysql.connector.connect(host='127.0.0.1', user='root', buffered=True)
mycursor = mydb.cursor()
mycursor.execute("DROP DATABASE IF EXISTS news")
mycursor.execute("CREATE DATABASE news")

In [55]:
# Drop the table of omdb if existed
mydb = mysql.connector.connect(host='127.0.0.1', user='root', buffered=True, db='news')
mycursor = mydb.cursor()
mycursor.execute("DROP TABLE IF EXISTS nasdaq_news")
mycursor.execute("""CREATE TABLE nasdaq_news
(company VARCHAR(7), 
titles VARCHAR(251),
release_date DATE,
day_of_week INT,
publisher VARCHAR(20),
page_url VARCHAR(124),
article LONGTEXT,
other_company_1 VARCHAR(7),
other_company_2 VARCHAR(7),
other_company_3 VARCHAR(7),
other_company_4 VARCHAR(7),
PRIMARY KEY (company, titles, page_url) 
)""")
mydb.commit()

In [56]:
# All the attributes we want to store in the database
col = ['company', 'titles', 'release_date', 'day_of_week', 'publisher', 'page_url', 'article', 'other_company_1', 'other_company_2', 'other_company_3', 'other_company_4']
# Insert data to the table
into = "INSERT INTO nasdaq_news(company, titles, release_date, day_of_week, publisher, page_url, article, other_company_1, other_company_2, other_company_3, other_company_4) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"

for i in notebook.tqdm(range(0, len(news_fulllist_wide))):
    news = news_fulllist_wide.iloc[i,:]
    if news['other company 1'] == 'None':
        other_company_1 = None
    elif news['other company 1'] is None:
        other_company_1 = None
    else:
        other_company_1 = news['other company 1']
        
    if news['other company 2'] == 'None':
        other_company_2 = None
    elif news['other company 2'] == '':
        other_company_2 = None
    else: 
        other_company_2 = news['other company 2']
        
    if news['other company 3'] == 'None':
        other_company_3 = None
    elif news['other company 3'] == '':
        other_company_3 = None
    else: 
        other_company_3 = news['other company 3']
    
    if news['other company 4'] == 'None':
        other_company_4 = None
    elif news['other company 4'] == '':
        other_company_4 = None
    else: 
        other_company_4 = news['other company 4']
        
    if news['release_date'] == 'NaT':
        release_date = None
    else:
        release_date = news['release_date']
        
    if news['weekday'] is None:
        weekday = None
    elif news['weekday'] is np.nan:
        weekday = None
    else:
        weekday = news['weekday'].item()
        
    value = (news['company'], news['titles'], release_date, weekday, news['publisher'], news['page_url'], news['article'], other_company_1, other_company_2, other_company_3, other_company_4)
    # Execute the insert command
    mycursor.execute(into, value)
    # Commit the execution
    mydb.commit()

print ('Record saved.')   
# Close the database
mydb.close()           


Record saved.


__Create a database for stock prices from APIs__

In [3]:
api_fulllist.head(2)

,date,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,company
0,2020-03-10,277.14,286.44,269.37,285.34,285.34,70721316,0.0,1,AAPL
1,2020-03-09,263.75,278.09,263.00,266.17,266.17,71686208,0.0,1,AAPL


Conform the company columns to the same as those in news list

In [16]:
news_fulllist['company'].unique()

array(['AAPL', 'AMZN', 'FB', 'TSLA', 'NFLX', 'GOOGL', 'ADBE', 'SBUX',
       'EXPEDIA', 'COSTCO'], dtype=object)

In [30]:
api_fulllist['company'].unique()

array(['AAPL', 'ADBE', 'AMZN', 'COSTCO', 'EXPEDIA', 'FB', 'GOOGL', 'NDAQ',
       'NFLX', 'SBUX', 'TSLA'], dtype=object)

In [57]:
api_fulllist.loc[api_fulllist.loc[:,'company'] == 'COST', 'company'] = 'COSTCO'
api_fulllist.loc[api_fulllist.loc[:,'company'] == 'EXPE','company'] = 'EXPEDIA'

In [58]:
# Drop the table of omdb if existed
mydb = mysql.connector.connect(host='127.0.0.1', user='root', buffered=True, db='news')
mycursor = mydb.cursor()
mycursor.execute("DROP TABLE IF EXISTS stock_prices")
mycursor.execute("""CREATE TABLE stock_prices
(date_of_price DATE, 
open_price DECIMAL,
high_price DECIMAL,
low_price DECIMAL,
close_price DECIMAL,
adjusted_price DECIMAL,
volume INT,
dividend_amount DECIMAL,
split_coef DECIMAL,
company VARCHAR(7),
PRIMARY KEY (company, date_of_price)
)""")
mydb.commit()

In [59]:
# # All the attributes we want to store in the database
# col = ['company', 'titles', 'release_date', 'day_of_week', 'publisher', 'page_url', 'article', 'other_company_1', 'other_company_2', 'other_company_3', 'other_company_4']
# Insert data to the table
into = "INSERT INTO stock_prices(date_of_price, open_price, high_price, low_price, close_price, adjusted_price, volume, dividend_amount, split_coef, company) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"

for i in notebook.tqdm(range(0, len(api_fulllist))):
    price = api_fulllist.iloc[i,:]
    value = (price['date'], price['1. open'].item(), price['2. high'].item(), price['3. low'].item(), price['4. close'].item(), price['5. adjusted close'].item(), price['6. volume'].item(), price['7. dividend amount'].item(), price['8. split coefficient'].item(), price['company'])
    # Execute the insert command
    mycursor.execute(into, value)
    # Commit the execution
    mydb.commit()

print ('Record saved.')   
# Close the database
mydb.close()           


Record saved.
